In [67]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [68]:
def calEnt(dataSet):
    n = dataSet.shape[0] #数据集总行数
    iSet = dataSet.iloc[:, -1].value_counts() #标签的所有类别
    # print("iSet=%s"  % (iSet))
    p = iSet / n #每一类标签所占比
    ent = (-p * np.log2(p)).sum() #计算信息熵
    return ent
 

In [69]:
#创建数据集
def createDataSet():
    row_data = {'no surfacing':[1,1,1,0,0],
                'flippers':[1,1,0,1,1],
                'fish':['yes','yes','no','no','no']}
    dataSet = pd.DataFrame(row_data)
    return dataSet

In [70]:
dataSet = createDataSet()
dataSet

,no surfacing,flippers,fish
0,1,1,yes
1,1,1,yes
2,1,0,no
3,0,1,no
4,0,1,no


In [71]:
print(calEnt(dataSet))



0.9709505944546686


In [72]:
#选择最优的列进行切分
def bestSplit(dataSet):
    baseEnt = calEnt(dataSet) #计算原始熵
    bestGain = 0 #初始化信息增益
    axis = -1 #初始化最佳切分列，标签列
    for i in range(dataSet.shape[1]-1): #对特征的每一列进行循环
        levels= dataSet.iloc[:,i].value_counts().index #提取出当前列的所有取值
        ents = 0 #初始化子节点的信息熵
        
        for j in levels: #对当前列的每一个取值进行循环
            childSet = dataSet[dataSet.iloc[:,i]==j] #某一个子节点的dataframe
            ent = calEnt(childSet) #计算某一个子节点的信息熵
            ents += (childSet.shape[0]/dataSet.shape[0])*ent #计算当前列的信息熵
            
        print(f'第{i}列的信息熵为{ents}')
        infoGain = baseEnt-ents #计算当前列的信息增益
        print(f'第{i}列的信息增益为{infoGain}')
        
        if (infoGain > bestGain):
            bestGain = infoGain #选择最大信息增益
            axis = i #最大信息增益所在列的索引
            
    return axis


In [73]:
bestSplit(dataSet)



第0列的信息熵为0.5509775004326937
第0列的信息增益为0.4199730940219749
第1列的信息熵为0.8
第1列的信息增益为0.17095059445466854


0

In [74]:
def mySplit(dataSet, axis, value):
    '''
    :param dataSet: 
    :param axis: 
    :param value: 
    :return: 
    #对dataSet的axis列值为value进行切分
    '''

    col = dataSet.columns[axis]
    redataSet = dataSet.loc[dataSet[col] == value, :].drop(col, axis=1)
    return redataSet



In [75]:
value =1
axis=0
mySplit(dataSet,axis,value)



,flippers,fish
0,1,yes
1,1,yes
2,0,no


In [76]:
col = dataSet.columns[axis]
col



'no surfacing'

In [77]:
dataSet.loc[dataSet[col] == value,:].drop(col,axis=1)


,flippers,fish
0,1,yes
1,1,yes
2,0,no


In [78]:
"""
函数功能：基于最大信息增益切分数据集，递归构建决策树
参数说明：
dataSet：原始数据集（最后一列是标签）
返回：
myTree：字典形式的树
"""
def createTree(dataSet):
    featlist = list(dataSet.columns) #提取出数据集所有的列
    classlist = dataSet.iloc[:,-1].value_counts() #获取最后一列类标签
    #判断最多标签数目是否等于数据集行数，或者数据集是否只有一列
    if classlist[0]==dataSet.shape[0] or dataSet.shape[1] == 1:
        return classlist.index[0] #如果是，返回类标签
    
    axis = bestSplit(dataSet) #确定出当前最佳切分列的索引
    bestfeat = featlist[axis] #获取该索引对应的特征
    myTree = {bestfeat:{}} #采用字典嵌套的方式存储树信息
    del featlist[axis] #删除当前特征
    
    valuelist = set(dataSet.iloc[:,axis]) #提取最佳切分列所有属性值
    for value in valuelist: #对每一个属性值递归建树
        myTree[bestfeat][value] = createTree(mySplit(dataSet,axis,value))
        
    return myTree



In [79]:
myTree = createTree(dataSet)
myTree



第0列的信息熵为0.5509775004326937
第0列的信息增益为0.4199730940219749
第1列的信息熵为0.8
第1列的信息增益为0.17095059445466854
第0列的信息熵为0.0
第0列的信息增益为0.9182958340544896


{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [80]:
# np.save('myTree.npy', myTree)
# read_myTree = np.load('myTree.npy').item()
# read_myTree


In [81]:
"""
函数功能：对一个测试实例进行分类
参数说明：
inputTree：已经生成的决策树
labels：存储选择的最优特征标签
testVec：测试数据列表，顺序对应原数据集
返回：
classLabel：分类结果
"""
def classify(inputTree, labels, testVec):
    firstStr = next(iter(inputTree)) #获取决策树第一个节点
    secondDict = inputTree[firstStr] #下一个字典
    featIndex = labels.index(firstStr) #第一个节点所在列的索引
    
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]) == dict :
                classLabel = classify(secondDict[key], labels, testVec)
            else:
                classLabel = secondDict[key]
            
    return classLabel


In [82]:
"""
函数功能：对测试集进行预测，并返回预测后的结果
参数说明：
train：训练集
test：测试集
返回：
test：预测好分类的测试集
"""
def acc_classify(train,test):
    inputTree = createTree(train) #根据测试集生成一棵树
    labels = list(train.columns) #数据集所有的列名称
    result = []
    
    for i in range(test.shape[0]): #对测试集中每一条数据进行循环
        testVec = test.iloc[i,:-1] #测试集中的一个实例
        classLabel = classify(inputTree,labels,testVec) #预测该实例的分类
        result.append(classLabel) #将分类结果追加到result列表中
        
    test['predict']=result #将预测结果追加到测试集最后一列
    acc = (test.iloc[:,-1]==test.iloc[:,-2]).mean() #计算准确率
    print(f'模型预测准确率为{acc}')
    
    return test

In [83]:
train = dataSet
test = dataSet.iloc[:3,:]
acc_classify(train,test)



第0列的信息熵为0.5509775004326937
第0列的信息增益为0.4199730940219749
第1列的信息熵为0.8
第1列的信息增益为0.17095059445466854
第0列的信息熵为0.0
第0列的信息增益为0.9182958340544896
模型预测准确率为1.0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,no surfacing,flippers,fish,predict
0,1,1,yes,yes
1,1,1,yes,yes
2,1,0,no,no


In [84]:
#导入相应的包
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import graphviz

#特征
Xtrain = dataSet.iloc[:,:-1]
#标签
Ytrain = dataSet.iloc[:,-1]
labels = Ytrain.unique().tolist()
Ytrain = Ytrain.apply(lambda x: labels.index(x)) #将本文转换为数字

#绘制树模型
clf = DecisionTreeClassifier()
clf = clf.fit(Xtrain, Ytrain)
tree.export_graphviz(clf)
dot_data = tree.export_graphviz(clf, out_file=None)
graphviz.Source(dot_data)

#给图形增加标签和颜色
dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=['no surfacing', 'flippers'],
                                class_names=['fish', 'not fish'],
                                filled=True, rounded=True,
                                special_characters=True)
graphviz.Source(dot_data)

#利用render方法生成图形
graph = graphviz.Source(dot_data)
graph.render("fish")

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'fish'], make sure the Graphviz executables are on your systems' PATH